# PIQ3: 测试数据集真值计算

In [ ]:
from piq_feature import ImSim
import pandas as pd
import numpy as np
from time import time
from helper import saveJson, loadJson

In [ ]:
imsim = ImSim(k=50)
train_data = pd.read_csv('data/train_dataset.csv')
test_data = pd.read_csv('data/test_dataset.csv')
piq_imdes = loadJson('data/piq_imdes.json')
piq_imdes = [np.array(des, dtype=np.uint8) for des in piq_imdes]

In [ ]:
test_repeats = list()

start = time()

for test_index in test_data.index:
    test_index = test_data.loc[test_index, 'index']
    test_des = piq_imdes[test_index]

    simto = None
    sim = 0
    for index in train_data['index']:
        train_des = piq_imdes[index]
        sim = imsim.calcSim(test_des, train_des)
        if sim > 0.01:
            print('sim between {} and {} is {}'.format(test_index, index, sim))
            simto = index
            break
    if simto is None:
        test_repeats.append((0, 0, sim))
    else:
        test_repeats.append((1, simto, sim))

print('Cost time = {}s'.format(time() - start))

In [ ]:
repeat, simto, sim = list(zip(*test_repeats))
df = pd.DataFrame(dict(repeat=repeat, simto=list(simto), sim=sim))
test_df = pd.concat([test_data, df], axis=1)

piq_hash = pd.read_csv('data/piq_imhash_k2.csv')

simto_hash = list()
index_hash = list()
for i in test_df.index:
    index, repeat, simto = test_df.loc[i, ['index', 'repeat', 'simto']]
    if repeat == 1:
        simto_hash.append(piq_hash.loc[simto, 'hash_k2'])
    else:
        simto_hash.append(0)
    index_hash.append(piq_hash.loc[index, 'hash_k2'])

test_df['hash'] = index_hash
test_df['simto_hash'] = simto_hash

In [ ]:
test_df.to_csv('data/piq_test1000_true.csv', index=False)